In [1]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize
from standard_plotting import set_size

In [2]:
indir_track_cascade_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/track_mu_Burnsample_RD_Full_db.csv"
indir_energy_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/energy_Burnsample_RD_Full_db.csv"
indir_zenith_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/zenith_Burnsample_RD_Full_db.csv"
indir_azimuth_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/azimuth_Burnsample_RD_Full_db.csv"
indir_multiclass_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/pid_Burnsample_RD_Full_db_new_muon.csv"
indir_position_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/['position_x', 'position_y', 'position_z']_Burnsample_RD_Full_db.csv"
pulses_strings_doms_RD_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/Pulses_doms_strings_times_burnsample.csv"
lvl3_variables_RD_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/lvl3_variables_burnsample.csv"

#Extra for MC: Retro, Oscweight

In [3]:
track_cascade_RD = pd.read_csv(indir_track_cascade_RD).sort_values('event_no').reset_index(drop = True)
energy_RD = pd.read_csv(indir_energy_RD).sort_values('event_no').reset_index(drop = True)
zenith_RD = pd.read_csv(indir_zenith_RD).sort_values('event_no').reset_index(drop = True)
azimuth_RD = pd.read_csv(indir_azimuth_RD).sort_values('event_no').reset_index(drop = True)
multiclass_RD = pd.read_csv(indir_multiclass_RD).sort_values('event_no').reset_index(drop = True)
position_RD = pd.read_csv(indir_position_RD).sort_values('event_no').reset_index(drop = True)
pulses_strings_doms_RD = pd.read_csv(pulses_strings_doms_RD_indir).sort_values('event_no').reset_index(drop = True)

useful_columns = ['event_no',
       'C2HR6', 'CausalVetoHits', 'CleanedFullTimeLength', 'DCFiducialHits',
       'L3_oscNext_bool', 'NAbove200Hits', 'NchCleaned', 'NoiseEngineNoCharge',
       'RTVetoCutHit', 'STW9000_DTW300Hits', 'UncleanedFullTimeLength',
       'VertexGuessZ', 'VetoFiducialRatioHits']

lvl3_variables_RD = pd.read_csv(lvl3_variables_RD_indir,usecols=useful_columns).sort_values('event_no').reset_index(drop = True)



In [4]:
def to_logit(p):
    eps = 0.0000001
    try:
        if np.isnan(p):
            return
        p = p*(1-2*eps)+eps
        logit = np.log(p/(1-p))
    except ZeroDivisionError as e:
        print(e)
    return logit

multiclass_RD['pid_neutrino_pred_logit'] = pd.Series(multiclass_RD['pid_neutrino_pred']).apply(to_logit)

In [6]:
print(np.shape(track_cascade_RD))
print(np.shape(energy_RD))
print(np.shape(zenith_RD))
print(np.shape(azimuth_RD))
print(np.shape(multiclass_RD))
#print(np.shape(lvl3_RD))
print(np.shape(position_RD))
print(np.shape(pulses_strings_doms_RD))
print(np.shape(lvl3_variables_RD))


(61739862, 3)
(61739862, 3)
(61739862, 4)
(61739862, 4)
(61739862, 7)
(61739862, 5)
(98212967, 5)
(61739862, 14)


In [14]:
print((track_cascade_RD.head(1)))
print((energy_RD.head(1)))
print((zenith_RD.head(1)))
print((azimuth_RD.head(1)))
print((multiclass_RD.head(1)))
print((position_RD.head(1)))
print((pulses_strings_doms_RD.head(1)))
print((lvl3_variables_RD.head(1)))


   track_mu_pred  event_no
0       0.433383       0.0
   energy_pred  event_no
0       3.8938       0.0
   zenith_pred  zenith_kappa  event_no
0     1.779149      2.967215       0.0
   azimuth_pred  azimuth_kappa  event_no
0       2.71805       0.320058       0.0
   pid_noise_pred  pid_muon_pred  pid_neutrino_pred  pid  event_no  \
0        0.894823       0.014967           0.090209 -1.0       0.0   

   pid_neutrino_pred_logit  
0                -2.311082  
   position_x_pred  position_y_pred  position_z_pred  event_no
0        63.402687       -109.64241      -194.517563       0.0
   event_no  N_pulses  N_string  N_doms  First_dom_time
0         0        31        22      28         16559.0
   event_no     C2HR6  CausalVetoHits  CleanedFullTimeLength  DCFiducialHits  \
0         0  0.666667             4.0                 2316.0             5.0   

   L3_oscNext_bool  NAbove200Hits  NchCleaned  NoiseEngineNoCharge  \
0              0.0            5.0         5.0                  0.0  

In [13]:
# track_cascade_RD.drop(columns='Unnamed: 0',inplace=True,inplace=True)
# print((energy_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((zenith_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((azimuth_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((multiclass_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((lvl3_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((position_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((pulses_strings_doms_RD.drop(columns='Unnamed: 0',inplace=True)))
# print((lvl3_variables_RD.drop(columns='Unnamed: 0',inplace=True)))

None
None
None
None
None


KeyError: "['Unnamed: 0'] not found in axis"

In [15]:
total_RD = pd.merge(track_cascade_RD.reset_index(drop=True),energy_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),zenith_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),azimuth_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),multiclass_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),position_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),pulses_strings_doms_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_RD = pd.merge(total_RD.reset_index(drop=True),lvl3_variables_RD.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)



In [16]:
print(total_RD.head(1))

   track_mu_pred  event_no  energy_pred  zenith_pred  zenith_kappa  \
0       0.433383       0.0       3.8938     1.779149      2.967215   

   azimuth_pred  azimuth_kappa  pid_noise_pred  pid_muon_pred  \
0       2.71805       0.320058        0.894823       0.014967   

   pid_neutrino_pred  ...  DCFiducialHits  L3_oscNext_bool  NAbove200Hits  \
0           0.090209  ...             5.0              0.0            5.0   

   NchCleaned  NoiseEngineNoCharge  RTVetoCutHit  STW9000_DTW300Hits  \
0         5.0                  0.0           1.0                 4.0   

   UncleanedFullTimeLength  VertexGuessZ  VetoFiducialRatioHits  
0                  10581.0   -268.549988                    0.0  

[1 rows x 32 columns]


In [17]:
Burnsample_weight = 1/(365.2425*24*3600*10*0.01)
total_RD['total_osc_weight'] = np.ones(len(total_RD))*Burnsample_weight

In [20]:
print(total_RD.columns)

Index(['track_mu_pred', 'event_no', 'energy_pred', 'zenith_pred',
       'zenith_kappa', 'azimuth_pred', 'azimuth_kappa', 'pid_noise_pred',
       'pid_muon_pred', 'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'N_pulses',
       'N_string', 'N_doms', 'First_dom_time', 'C2HR6', 'CausalVetoHits',
       'CleanedFullTimeLength', 'DCFiducialHits', 'L3_oscNext_bool',
       'NAbove200Hits', 'NchCleaned', 'NoiseEngineNoCharge', 'RTVetoCutHit',
       'STW9000_DTW300Hits', 'UncleanedFullTimeLength', 'VertexGuessZ',
       'VetoFiducialRatioHits', 'total_osc_weight'],
      dtype='object')


In [21]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"

total_RD.to_csv(outdir + 'Burnsample_1_percent_all_events_all_variables.csv',index=False)

In [24]:
np.shape(total_RD[total_RD['pid_neutrino_pred_logit']>1])

(966537, 33)

In [25]:
small_RD = total_RD[total_RD['pid_neutrino_pred_logit']>1]
small_RD.to_csv(outdir + 'Burnsample_1_percent_neutrino_prob_logit_above_1_all_variables.csv',index=False)

In [4]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"


In [2]:
test = pd.read_csv("/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/Burnsample_1_percent_all_events_all_variables.csv")

In [5]:
small_small_RD = test[test['pid_neutrino_pred_logit']>12]
small_small_RD.to_csv(outdir + 'Burnsample_1_percent_neutrino_prob_logit_above_12_all_variables.csv',index=False)

In [6]:
print(np.shape(small_small_RD))

(18226, 33)
